<a href="https://colab.research.google.com/github/josegoisgit/dadosgov/blob/main/query_dadosgov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Módulos e pacotes

## Instalação

In [39]:
!pip install unidecode

## Importação

In [40]:
import re
import requests
import pandas

import unidecode


from itertools import chain
from lxml.etree import HTML

import IPython
import ipywidgets
import google.colab

# Funções

## auxiliares

In [41]:

strip = lambda X: ('\n'.join(set(x.strip() for x in X if x.strip()))).strip()
plain = lambda i: list(chain.from_iterable(i))


## get_url_html

In [42]:
def get_url_html(url):
    requested         = requests.request('GET', url)
    html              = HTML(requested.text)    
    html.nsmap['url'] = url

    return html


In [43]:
#Exemplo:
html = get_url_html('https://docs.python.org')
html.xpath('.//a/@href')

['genindex.html',
 'py-modindex.html',
 'https://www.python.org/',
 '#',
 'whatsnew/3.9.html',
 'whatsnew/index.html',
 'tutorial/index.html',
 'library/index.html',
 'reference/index.html',
 'using/index.html',
 'howto/index.html',
 'installing/index.html',
 'distributing/index.html',
 'extending/index.html',
 'c-api/index.html',
 'faq/index.html',
 'py-modindex.html',
 'genindex.html',
 'glossary.html',
 'search.html',
 'contents.html',
 'bugs.html',
 'https://devguide.python.org/docquality/#helping-with-documentation',
 'about.html',
 'license.html',
 'copyright.html',
 'download.html',
 'https://docs.python.org/3.10/',
 'https://docs.python.org/3.9/',
 'https://docs.python.org/3.8/',
 'https://docs.python.org/3.7/',
 'https://docs.python.org/3.6/',
 'https://docs.python.org/3.5/',
 'https://docs.python.org/2.7/',
 'https://www.python.org/doc/versions/',
 'https://www.python.org/dev/peps/',
 'https://wiki.python.org/moin/BeginnersGuide',
 'https://wiki.python.org/moin/PythonBooks',


## xpath

### contains_clause

In [44]:
def contains_clause(dic, operator='OR'):
    predicados = ["contains(@{k},'{v}')".format(k=k,v=v) for k,v in dic.items()]

    return operator.join(predicados)


### compose_xpath

In [45]:

def compose_xpath(element,attribute,content):
    dictionary = contains_clause({attribute:content})

    str_dict = {
        'element' : element,
        'dictionary' : dictionary 
    }

    return "{element}[{dictionary}]".format(**str_dict)


### get_xnodes

In [46]:

def get_xnodes(html, element, attribute='class', content='', complement=''):
    if complement:
        xnode = html.xpath(compose_xpath(element,attribute,content) + complement)
    else:
        xnode = html.xpath(compose_xpath(element,attribute,content) )
    return xnode if xnode else None


### get_xnode

In [47]:

def get_xnode(html, element, attribute='class', content='', complement=''):
    xnodes = get_xnodes(html, element, attribute, content, complement)
    return xnodes[0] if xnodes else None


## datasets

### get_html_page_count

In [48]:

def get_html_page_count(html):
    xnode = get_xnode( html, '//div', 'class', 'pagination' ) 
    
    if not xnode:
        return 1
    
    pdiv = xnode.xpath( '//li/a/text()' )
    
    def pmax(pdiv):
        V = [1]
        for p in pdiv:
            try:
                v = int(p)
                V.append(v)       
            except:
                continue
        return max(V)    
    return pmax(pdiv)


### get_query_page_count

In [49]:

def get_query_page_count(hyperlink,query):
    html = get_url_html( hyperlink + '/dataset?q=%s' % query)
    return get_html_page_count( html )


### get_html_items_href






In [50]:

def get_html_items_href(html):
    xnodes = get_xnodes( html, './/h3', 'class', 'dataset-heading' , '//a/@href' ) 
    return xnodes


### get_html_dataset

In [51]:

def get_html_dataset(html):    
    article_html = get_xnode(html , '//article')
    
    dataset = dict()
    
    dataset['organization'] = ' '.join([x.strip() for x in get_xnodes(html, './/section','class','module-content','//h1//text()')])
    dataset['title']        = strip( get_xnodes( article_html , 'div/h1/text()') ) 
    dataset['key']          = re.sub('[^\w]','_',unidecode.unidecode(dataset['title']).lower())
    z = get_xnodes(article_html,'//div','class','notes','//p/text()')
    dataset['text'] = strip(z) if z else ''
    dataset['url']          = get_xnodes( article_html, '//ol', 'class', 'breadcrumb', '//li/a/@href')

    return dataset


### get_html_dataset_license

In [52]:

def get_html_dataset_license(html,hyperlink='https://dados.gov.br'):
    xnode = get_xnode( html, '//section','class','license') 

    if not xnode: return dict(title='NA',href='',text='',logo='')

    license_title = xnode.xpath( './/@title' )
    license_href  = xnode.xpath( './/@href')
    license_text  = [ text.strip() for text in xnode.xpath( './/text()' ) if text.strip() ]
    license_logo  = [ hyperlink + p for p in xnode.xpath( './/img//@src' ) ]
    
    license_dict          = dict()
    license_dict['title'] = license_title
    license_dict['href']  = license_href
    license_dict['text']  = license_text
    license_dict['logo']  = license_logo

    return license_dict


### get_html_dataset_resources_href

In [53]:

def get_html_dataset_resources_href(html):
    return get_xnodes(html,'//a','class','heading','/@href')


### get_html_resource_info





In [54]:

def get_html_resource_info(html):
    
    ths     = get_xnodes( html, '//table', 'class', 'table-condensed','/tbody//th/text()')
    headers = [re.sub('[^\w]','_',unidecode.unidecode(th.lower())) for th in ths]
    tds     = get_xnodes( html, '//table', 'class', 'table-condensed','/tbody//td')
    data    = [strip(get_xnodes(td, './/text()')).strip() for td in tds]

    info_dict        = {h:d for h,d in zip(headers,data)}
    info_dict['url'] = get_xnode( html, '//a','class','resource-url-analytic','/@href')    

    return info_dict

### auxiliares

In [55]:

def __dir_repr__(variable=dict,pattern=''):
    dir_type_dict = dict()
    
    for attribute in dir(variable):
        if not re.findall(pattern, attribute): continue
        value = getattr(variable,attribute)
        typename = type(value).__name__
        
        if typename not in dir_type_dict:
            dir_type_dict[typename] = [(attribute,value)]
        else:
            dir_type_dict[typename].append((attribute,value))
        
    for key, items in dir_type_dict.items():
        print(key)
        print()
        for attribute,value in items:
            print('\t',attribute)
        print()
        #print('{: <30s}{:<20s}'.format(typename, attribute))


In [56]:

def show_url( url='https://dados.gov.br'):
    dadosgov_frame = IPython.display.IFrame(src=url,width='100%',height='500px')
    display(dadosgov_frame)
    return


In [57]:

def query_site( query=''):
    query = query.lower()
    query = unidecode.unidecode(query)
    query = re.sub('[^a-z\s\'\"]','',query)
    query = re.sub('\s+','+',query)
    
    url = 'https://dados.gov.br/dataset?q=%s' % query
    dadosgov_frame = IPython.display.IFrame(src=url,width='100%',height='500px')
    display(dadosgov_frame)
    return dadosgov_frame



In [58]:

get_query_page_url   = lambda h, q, p : h + '/dataset' + '?q=' + q + '&page=' + str(p)
get_query_page_html  = lambda h, q, p : get_url_html( get_query_page_url( h, q, p ) )
get_query_page_items = lambda h, q, p : get_html_items_href( get_query_page_html( h, q, p ) )
get_query_items_href = lambda h, q    : plain([ get_query_page_items( h, q, p+1 ) for p in range( get_query_page_count( h, q ) ) ])


# Leitura

## argumentos

In [59]:
query     = 'UFRN discentes'
hyperlink = 'https://dados.gov.br'

In [60]:
qri_label = ipywidgets.widgets.Label('Query')
qri_text  = ipywidgets.widgets.Text(query)

In [61]:
qri_box = ipywidgets.widgets.HBox([qri_label, qri_text])

In [62]:
display(ipywidgets.widgets.VBox([qri_box]))

In [63]:
query     = qri_text.value

In [64]:
query_url = get_query_page_url(hyperlink,query,1)

In [65]:
show_url(query_url)

## visualização

In [66]:
get_xnode( html, './/class','pagination')

In [67]:
resources = []
items_href = get_query_items_href(hyperlink,query)

In [68]:
ih = 0

print(hyperlink + items_href[ih])
show_url(hyperlink + items_href[ih])

https://dados.gov.br/dataset/discentes


## acesso

In [69]:
import time

In [70]:
import pandas

for item_href in items_href:
    item_url = hyperlink + item_href

    print('\n\n\t', item_url, end='\n\n')
    dataset_html   = get_url_html( item_url )

    dataset        = get_html_dataset( dataset_html )
    license        = get_html_dataset_license ( dataset_html )
    resources_href = get_html_dataset_resources_href( dataset_html )

    for resource_href in resources_href:
        resource_url  = hyperlink + resource_href 

        resource_html = get_url_html( resource_url )  
        resource_dict = get_html_resource_info(resource_html)

        a = 'http://landpage-h.cgu.gov.br/dadosabertos/index.php?url='

        if resource_dict['url']:
            resource_dict['url_out'] = resource_dict['url'].replace(a,'')
        else:
            resource_dict['url_out'] = None
        resource_dict['dataset'] = dataset.copy()
        resource_dict['license'] = license

        resource_dict['havested'] = time.ctime()
        
        print('\t\t',resource_dict['url_out'], end='\n')

        resources.append( resource_dict )
    




	 https://dados.gov.br/dataset/discentes



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  """


		 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/a55aef81-e094-4267-8643-f283524e3dd7/download/discentes-2019.csv
		 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/146b749b-b9d0-49b2-b114-ac6cc82a4051/download/discentes-2018.csv
		 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/dc732572-a51a-4d4a-a39d-2db37cbe5382/download/discentes-2017.csv
		 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/7d2fa5b3-743f-465f-8450-91719b34a002/download/discentes-2016.csv
		 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/e2b5b843-4f58-497e-8979-44daf8df8f94/download/discentes-2015.csv
		 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/6c23a430-9a7c-4d0f-9602-1d5d97d40e6a/download/discentes-2014.csv
		 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/dba208c2-822f-4e26-adc3-b61d4cb110b6/download/discentes-2013.csv

In [71]:
for r,resource in enumerate(resources):
    d2 = resources[r].copy()
    for field,value in sorted(resource.items(),key=lambda e: e[0]):
        value = d2[field]
        if isinstance(value, dict):
            for key, door in value.items():
                d2[field+'_'+key] = door
            d2.pop(field)
    resources[r] = d2

# Dados

In [72]:
df_datagov = pandas.DataFrame(resources)

In [73]:
df_datagov = df_datagov[sorted(df_datagov.keys())]

In [74]:
df_datagov.iloc[5]

created                                                  há mais de 3 anos
criado                                                     27/Outubro/2017
dataset_key                                                      discentes
dataset_organization     Universidade Federal do Rio Grande do Norte - ...
dataset_text                                Relação dos discentes da UFRN.
dataset_title                                                    Discentes
dataset_url              [/, /organization, /organization/universidade-...
datastore_active                                                      True
format                                                                 CSV
formato                                                                CSV
havested                                          Sun Apr 25 07:48:07 2021
id                                    6c23a430-9a7c-4d0f-9602-1d5d97d40e6a
last_modified                                                12 dias atrás
licenca                  

In [75]:
df_datagov.to_csv('./resultado.csv')
google.colab.files.download('./resultado.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## texto

In [76]:
for a,b in df_datagov.groupby('package_id'):
    print(b.iloc[0].dataset_key,a,end='\n\n')

    for url in b.url_out.values:
        print('\t',url)
    print()
    

discentes 554c2d41-cfce-4278-93c6-eb9aa49c5d16

	 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/a55aef81-e094-4267-8643-f283524e3dd7/download/discentes-2019.csv
	 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/146b749b-b9d0-49b2-b114-ac6cc82a4051/download/discentes-2018.csv
	 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/dc732572-a51a-4d4a-a39d-2db37cbe5382/download/discentes-2017.csv
	 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/7d2fa5b3-743f-465f-8450-91719b34a002/download/discentes-2016.csv
	 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/e2b5b843-4f58-497e-8979-44daf8df8f94/download/discentes-2015.csv
	 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/6c23a430-9a7c-4d0f-9602-1d5d97d40e6a/download/discentes-2014.csv
	 http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/dba208c2-822f-4e26-adc3